In [65]:
import angr
from angr import sim_options as so
import claripy
from angrutils import plot_cfg
import os

In [103]:
# binary = '/home/bupt1112/data/bctf/popcalc_001/bin/cb'
# binary = '/home/bupt1112/data/bctf/crash_001/bin/cb'
# binary = '/home/bupt1112/codes/myUtilities/angr_exit'
binary = '/home/bupt1112/codes/myUtilities/angr__exit'
# binary = '/home/bupt1112/codes/myUtilities/angr_return'
p = angr.Project(binary, auto_load_libs=False)

In [104]:
main = p.loader.main_object.get_symbol("main")
print(hex(main.rebased_addr))
print(hex(p.entry))

0x804846b
0x8048370


In [105]:
cfg = p.analyses.CFGEmulated(fail_fast=True, starts=[main.rebased_addr])
# cfg = p.analyses.CFGFast()

In [106]:
plot_cfg(cfg, os.path.basename(binary), asminst=True, remove_imports=True, remove_path_terminator=True)

In [107]:
# es = p.factory.entry_state(add_options={so.REVERSE_MEMORY_NAME_MAP}, stdin=angr.SimFileStream)
# sm = p.factory.simulation_manager(es, save_unconstrained=True)
es = p.factory.full_init_state()
sm = p.factory.simulation_manager(es, save_unconstrained=True)

In [108]:
states = []
def overflow_filter(simgr):
#     if len(simgr.active) > 0:
#         print(simgr.active[0].regs.ip)
    for path in simgr.unconstrained:

        state = path.state

        eip = state.regs.pc
        bits = state.arch.bits
        state_copy = state.copy()

        # Constrain pc to 0x41414141 or 0x41414141414141 'A'
        constraints = []
        for i in range(bits / 8):
            curr_byte = eip.get_byte(i)
            constraint = claripy.And(curr_byte == 0x41)
            constraints.append(constraint)

        # Check satisfiability
        if state_copy.se.satisfiable(extra_constraints=constraints):
            for constraint in constraints:
                state_copy.add_constraints(constraint)

            # Check by input
            stdin_str = str(state_copy.posix.dumps(0).replace('\x00', '').replace('\x01', ''))
            print('[+] {}'.format(stdin_str))
            if 'A' in stdin_str:

                # Constrain EIP to 0x41414141 or 0x4141414141414141
                constraints = []
                for i in range(bits / 8):
                    curr_byte = eip.get_byte(i)
                    constraint = claripy.And(curr_byte == 0x41)
                    constraints.append(constraint)

                # Constrain STDIN to printable if we can
                if state.se.satisfiable(extra_constraints=constraints):
                    for constraint in constraints:
                        state.add_constraints(constraint)

                # Constrain rest of input to be printable
                stdin = state.posix.files[0]
                constraints = []
                #stdin_size = len(stdin.all_bytes())
                stdin_size = 300
                stdin.length = stdin_size
                stdin.seek(0)
                stdin_bytes = stdin.all_bytes()
                for i in range(stdin_size):
                    curr_byte = stdin.read_from(1)
                    constraint = claripy.And(urr_byte > 0x2F, curr_byte < 0x7F)
                    if state.se.satisfiable(extra_constraints=[constraint]):
                        constraints.append(constraint)

                # Constrain STDIN to printable if we can
                if state.se.satisfiable(extra_constraints=constraints):
                    for constraint in constraints:
                        state.add_constraints(constraint)

                # Get the string coming into STDIN
                stdin_str = repr(str(state.posix.dumps(0).replace('\x00', '').replace('\x01', '')))
                print("[+] Vulnerable path found {}".format(stdin_str))
                state.globals['type'] = "Overflow"
                simgr.stashes['found'].append(path)
                simgr.stashes['unconstrained'].remove(path)

    return simgr

end_state = None

sm.explore(find=lambda s: 'type' in s.globals, step_func=overflow_filter)
if 'found' in sm.stashes and len(sm.found):
    end_state = sm.found[0]
else:
    print('[!] No found.')

WARNING | 2020-06-29 19:39:59,071 | angr.state_plugins.symbolic_memory | Filling register edi with 4 unconstrained bytes referenced from 0x80484b1 (__libc_csu_init+0x1 in angr__exit (0x80484b1))
WARNING | 2020-06-29 19:39:59,074 | angr.state_plugins.symbolic_memory | Filling register ebx with 4 unconstrained bytes referenced from 0x80484b3 (__libc_csu_init+0x3 in angr__exit (0x80484b3))
WARNING | 2020-06-29 19:39:59,171 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffefffc with 67 unconstrained bytes referenced from 0x8100008 (puts+0x0 in extern-address space (0x8))
WARNING | 2020-06-29 19:39:59,172 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffeff40 with 4 unconstrained bytes referenced from 0x8100008 (puts+0x0 in extern-address space (0x8))
WARNING | 2020-06-29 19:39:59,232 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffeff24 with 4 unconstrained bytes referenced from 0x80484c2 (__libc_csu_init+0x12 in angr__exit (0x80484c2))
WARNING | 202

[!] No found.


In [109]:
sm

<SimulationManager with all stashes empty (1 errored)>

In [110]:
sm.errored[0]

<State errored with "No bytes in memory for block starting at 0x0.">

In [73]:
[b for b in sm.errored[0].state.history.actions]

[<SimActionData LinuxLoader() reg/write>,
 <SimActionData LinuxLoader() reg/write>,
 <SimActionData LinuxLoader() reg/write>,
 <SimActionData LinuxLoader() reg/write>,
 <SimActionData __libc_start_main() reg/read>,
 <SimActionData __libc_start_main() mem/read>,
 <SimActionData __libc_start_main() reg/read>,
 <SimActionData __libc_start_main() mem/read>,
 <SimActionData __libc_start_main() reg/read>,
 <SimActionData __libc_start_main() mem/read>,
 <SimActionData __libc_start_main() reg/read>,
 <SimActionData __libc_start_main() mem/read>,
 <SimActionData __libc_start_main() reg/read>,
 <SimActionData __libc_start_main() mem/read>,
 <SimActionData __libc_start_main() reg/read>,
 <SimActionData __libc_start_main() reg/read>,
 <SimActionData __libc_start_main() reg/read>,
 <SimActionData __libc_start_main() reg/read>,
 <SimActionData __libc_start_main() reg/read>,
 <SimActionData __libc_start_main() reg/read>,
 <SimActionData __libc_start_main() reg/read>,
 <SimActionData __libc_start_main